# Modelo NLP: Entrenamiento

In [5]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_4/part_05'

#### Subimos los datos a s3

In [6]:
train_channel = prefix + '/train'
sess.upload_data(path='dbpedia_csv/dbpedia.train', bucket=bucket, key_prefix=train_channel)

's3://sagemaker-eu-west-1-467432373215/module_4/part_05/train/dbpedia.train'

In [7]:
validation_channel = prefix + '/validation'
sess.upload_data(path='dbpedia_csv/dbpedia.validation', bucket=bucket, key_prefix=validation_channel)

's3://sagemaker-eu-west-1-467432373215/module_4/part_05/validation/dbpedia.validation'

#### Modelo

- Usaremos el modelo https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/blazingtext.html

- Amazon SageMaker BlazingText proporciona implementaciones altamente optimizadas de los algoritmos de Word2VEC y de clasificación de texto. El algoritmo Word2vec es útil para muchas tareas de procesamiento de lenguaje natural (NLP) posteriores, como por ejemplo, análisis del sentimiento de reconocimiento de entidad nombrada, traducción automática, etc. La clasificación de texto es una tarea importante para las aplicaciones que realizan búsquedas web, recuperación de información, funciones de clasificación y clasificación de documentos.

- El algoritmo Word2vec asigna palabras a vectores distribuidos de alta calidad. La representación vectorial resultante de una palabra se denomina una incrustación de palabra. Las palabras similares desde el punto de vista semántico corresponden a vectores que se acercan entre sí. De esta forma, las incrustaciones de palabras capturan relaciones semánticas entre palabras. 

In [8]:
image=sagemaker.image_uris.retrieve(framework='blazingtext', 
                                    region=region, 
                                    version='1')
print(image)

685385470294.dkr.ecr.eu-west-1.amazonaws.com/blazingtext:1


In [9]:
s3_output_location = f's3://{bucket}/{prefix}/output'

estimator = sagemaker.estimator.Estimator(
        image,
        role,
        instance_count=1,
        instance_type='ml.c5.2xlarge',
        volume_size=30,
        max_run=360000,
        input_mode='File',
        enable_sagemaker_metrics=True,
        output_path=s3_output_location,
        hyperparameters={
            'mode': 'supervised',
            'epochs': 20,
            'min_count': 2,
            'learning_rate': 0.05,
            'vector_dim': 10,
            'early_stopping': True,
            'patience': 4,
            'min_epochs': 5,
            'word_ngrams': 2,
        },
)

#### Entrenamiento

In [11]:
s3_train_data = f's3://{bucket}/{train_channel}'
s3_validation_data = f's3://{bucket}/{validation_channel}'
print(s3_train_data)
print(s3_validation_data)

data_channels = {'train': s3_train_data, 
                 'validation': s3_validation_data}

jobname = f'dbpedia-blazingtext'

s3://sagemaker-eu-west-1-467432373215/module_4/part_05/train
s3://sagemaker-eu-west-1-467432373215/module_4/part_05/validation


In [12]:
estimator.fit(inputs=data_channels,
              job_name=jobname,
              logs=True)

2022-12-13 21:48:35 Starting - Starting the training job...
2022-12-13 21:48:58 Starting - Preparing the instances for trainingProfilerReport-1670968115: InProgress
......
2022-12-13 21:50:03 Downloading - Downloading input data...
2022-12-13 21:50:19 Training - Training image download completed. Training in progress.Arguments: train
[12/13/2022 21:50:20 WARNING 140110651049792] Loggers have already been setup.
[12/13/2022 21:50:20 WARNING 140110651049792] Loggers have already been setup.
[12/13/2022 21:50:20 INFO 140110651049792] nvidia-smi took: 0.025187015533447266 secs to identify 0 gpus
[12/13/2022 21:50:20 INFO 140110651049792] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[12/13/2022 21:50:20 INFO 140110651049792] Processing /opt/ml/input/data/train/dbpedia.train . File size: 34.936402320861816 MB
[12/13/2022 21:50:20 INFO 140110651049792] Processing /opt/ml/input/data/validation/dbpedia.validation . File siz

- Podemos acceder al modelo con estimator.model_data
- Este modelo lo podemos desplegar como veremos más adelante o descargarlo en nuestro ordenador y usar el programa o SDK de fastText (https://fasttext.cc/).

In [14]:
estimator.model_data

's3://sagemaker-eu-west-1-467432373215/module_4/part_05/output/dbpedia-blazingtext/output/model.tar.gz'

In [ ]:
!aws s3 cp {estimator.model_data} ./dbpedia_csv/

In [ ]:
%%sh
cd dbpedia_csv/
tar -zxf model.tar.gz